In [10]:
import os
import pandas as pd 
import numpy as np
import os, shutil, glob
from random import randint
import re, math
import datetime
import gc
from pathlib import Path

os.getcwd()


'/scratch2-shared/david/colopaint3D/notebooks'

### Read data

In [11]:
# pd.set_option("max_colwidth", 2000)
# pd.set_option("display.max_columns", 1000)
# pd.set_option("display.max_rows", 1000)

# dirlist = ['/share/data/cellprofiler/automation/results/spher010-DLD1-L5/3958/5244/'
#            #, '/share/data/cellprofiler/automation/results/spher010-HCT116-L6/3943/5241/'
#            , '/share/data/cellprofiler/automation/results/spher010-HCT15-L2/3954/5246/'
#            , '/share/data/cellprofiler/automation/results/spher010-HT29-L4/3962/5247/']
# BaseDir = '/share/data/analyses/christa/colopaint3D/pilots1-10/results_spheroid'
rootDir = '/share/data/cellprofiler/automation/results/'
# BaseImgDir = '/share/mikro2/nikon/spheroid-test/pilot12-HCT116-day2/'
# BaseDir = '/share/data/cellprofiler/automation/results/pilot12-HCT116-day2/3797/5112'
# sorted(os.listdir(dirlist[0]))

### Input directories

In [12]:
OutputDir = 'output/spher_colo52_v1/1_FeaturesImages'
if not os.path.exists(OutputDir): 
    os.makedirs(OutputDir)
NameContains = ''
InputFolders = pd.read_csv('./settings/filemap.csv')
print(InputFolders )
            


       feature_names
0     featICF_nuclei
1      featICF_cells
2  featICF_cytoplasm


In [13]:
now = datetime.datetime.now()
print ('Current date and time : ')
print (now.strftime('%Y-%m-%d %H:%M:%S'))

Current date and time : 
2023-11-22 22:09:13


In [14]:
mdf = pd.read_csv('/home/jovyan/scratch2-shared/david/colopaint3D/notebooks/settings/spher-colo52-v1-import-inputFiles-and-PLAIDresults.csv', sep=',')

barcodes = mdf['barcode'].unique()
# metaEx = metaEx[~(metaEx['Metadata_Barcode'] == df['Metadata_Barcode'].unique()[0])]

In [15]:
barcodes


array(['PB000137', 'PB000139', 'PB000141', 'PB000138', 'PB000140',
       'PB000142'], dtype=object)

In [16]:
mdf_op = mdf[(mdf['barcode'] == barcodes[0])]

In [17]:
mdf_op['cp_id'].unique()

array([5532])

In [18]:
# medianImageAllPlates = pd.DataFrame()
# meanImageAllPlates = pd.DataFrame()
# wellmismatch = 0
df_scall = pd.DataFrame()
df_meanall = pd.DataFrame()
df_medianall = pd.DataFrame()
# DataFrameDictionary = {}s
ObjectList = ['featICF_nuclei', 'featICF_cells', 'featICF_cytoplasm']
for BaseDir in barcodes:
    mdf_op = mdf[(mdf['barcode'] == BaseDir)]
    image_id = mdf_op['image_id'].unique()[0]
    cp_id = mdf_op['cp_id'].unique()[0]


    nuclei = pd.read_parquet(rootDir+f'{BaseDir}/{image_id}/{cp_id}/{ObjectList[0]}.parquet').add_prefix('Nuclei_').reset_index()
    cytoplasm = pd.read_parquet(rootDir+f'{BaseDir}/{image_id}/{cp_id}/{ObjectList[1]}.parquet').add_prefix('Cytoplasm_').reset_index()
    cells = pd.read_parquet(rootDir+f'{BaseDir}/{image_id}/{cp_id}/{ObjectList[2]}.parquet').add_prefix('Cells_').reset_index()

    # print(nuclei.shape)
    # print(cytoplasm.shape)
    # print(cells.shape)

    # step 1: Take the mean values of multiple nuclei belonging to one cell
    nuclei = nuclei.groupby(["Nuclei_Metadata_Barcode","Nuclei_Metadata_Well","Nuclei_Parent_cells", 'Nuclei_Metadata_Site'], as_index=False).mean(numeric_only=True)
    # nuclei = nuclei.groupby(["Nuclei_Metadata_Barcode","Nuclei_Metadata_Well","Nuclei_Parent_cells", 'Nuclei_Metadata_Site'], as_index=False).mean(numeric_only=True)
    # step 2: merge nuclei and cytoplasm objects
    df = pd.merge(nuclei, cytoplasm, how='left', left_on=['Nuclei_Metadata_Well', 'Nuclei_Metadata_Site',"Nuclei_Parent_cells", 'Nuclei_Metadata_Barcode'],
                    right_on = ['Cytoplasm_Metadata_Well','Cytoplasm_Metadata_Site', "Cytoplasm_ObjectNumber", 'Cytoplasm_Metadata_Barcode'])
    # step 3: join cells objects
    df = pd.merge(df, cells, how='left', left_on=['Cytoplasm_Metadata_Well','Cytoplasm_Metadata_Site',"Cytoplasm_ObjectNumber", 'Cytoplasm_Metadata_Barcode'],
                    right_on = ['Cells_Metadata_Well','Cells_Metadata_Site',"Cells_ObjectNumber", 'Cells_Metadata_Barcode'])


    print(f'Shape Pre Drops{df.shape}')
    df.dropna(axis=0, inplace=True)
    print(f'Shape Post Drops{df.shape}')

    #------------------- format feature names to BROAD names --------------------------------#
    first_column = df.pop('Cytoplasm_Metadata_Barcode')
    second_column = df.pop('Cytoplasm_Metadata_Well')
    third_column = df.pop('Cytoplasm_Metadata_Site')

    df.insert(0, 'Metadata_Barcode', first_column)
    df.insert(1, 'Metadata_Well', second_column)
    df.insert(2, 'Metadata_Site', third_column)
    nameargs = ['Nuclei', 'Cytoplasm', 'Cells']
    for name in nameargs:
        df = df.loc[:,~df.columns.str.contains(name+'_Metadata_', case=True)]
    # df = df.loc[:,~df.columns.str.contains(name+'_Location_Center_', case=True)]


    # ------------------ add image indentifyer for metdata and QC -----------------------------#
    df = df.loc[:,~df.columns.str.contains('Unnamed|index|Cytoplasm_Meta|Cells_Meta|Nuclei_Meta|FileName|PathName|_ImageNumber')]

    NrOfObjects = df.shape[0]
    # print(df['Metadata_Well'])
    # Wells = sorted(set(df['Metadata_Well']))
    Wells = sorted(list(set(df['Metadata_Well'])))
    NrOfWells = len(Wells)
    Rows = sorted(list(set([w[0] for w in Wells])))
    print(*Rows)
    NrOfRows = len(Rows)
    Columns = sorted(list(set([w[1:] for w in Wells])))
    NrOfColumns = len(Columns)
    print(*Columns)
    print('Plate complete: {}'.format(NrOfRows*NrOfColumns==NrOfWells))

    AllWells =[]
    for R in Rows:
        for C in Columns:
            RC = R + C
            # print(RC)
            AllWells += [RC]
    # print(len(AllWells))
    print('Number of wells: {} (full plate is {})'.format(NrOfWells, len(AllWells)))
    # del nuclei
    # del cytoplasm
    # del cells
    dfdb = df.head()
    numeric_df = df.select_dtypes(include=[np.number])

    ### Clipping Outliers, NaNs, and Infs
    medians = numeric_df.median(skipna=True)
    psu = numeric_df.apply(lambda x: np.nanpercentile(x[x != np.inf], 99), axis=0)
    psl = numeric_df.apply(lambda x: np.nanpercentile(x[x != np.inf], 1), axis=0)
    # numeric_df.fillna(medians, inplace=True)

    numeric_df.replace(np.inf, psu, inplace=True)
    numeric_df.clip(upper=psu, lower=psl, axis=1, inplace=True)
    df[numeric_df.columns] = numeric_df
    df = df.copy()
    # print(df.)
    
    ### Adding additional Metadata
    target1 = pd.DataFrame()
    target2 = pd.DataFrame()
    target3 = pd.DataFrame()
    target4 = pd.DataFrame()
    target5 = pd.DataFrame()
    # metaEx = pd.read_csv('/home/jovyan/scratch2-shared/david/colopaint3D/notebooks/settings/spher010-metadata.csv', sep=',')

    # barcode = df['Metadata_Barcode'].unique()[0]
    # metaEx = metaEx[~(metaEx['Metadata_Barcode'] == df['Metadata_Barcode'].unique()[0])]
    metaEx = mdf_op
    target1['Metadata_Cmpd'] = df['Metadata_Well'].apply(lambda x: metaEx.loc[metaEx['well_id'] == x, 'cmpdname'].values[0])
    df.insert(3, 'Metadata_Cmpd', target1['Metadata_Cmpd'])
    target2['Metadata_Line'] = df['Metadata_Well'].apply(lambda x: metaEx.loc[metaEx['well_id'] == x, 'cell_line'].values[0])
    df.insert(4, 'Metadata_Line', target2['Metadata_Line'])
    target3['Metadata_Moa'] = df['Metadata_Well'].apply(lambda x: metaEx.loc[metaEx['well_id'] == x, 'pathway'].values[0])
    df.insert(5, 'Metadata_Moa', target3['Metadata_Moa'])
    target4['Metadata_InChiKey'] = df['Metadata_Well'].apply(lambda x: metaEx.loc[metaEx['well_id'] == x, 'inkey'].values[0])
    df.insert(5, 'Metadata_InChiKey', target4['Metadata_InChiKey'])
    target4['Metadata_Target'] = df['Metadata_Well'].apply(lambda x: metaEx.loc[metaEx['well_id'] == x, 'target'].values[0])
    df.insert(5, 'Metadata_Target', target4['Metadata_Target'])
    
    df = df.copy()
    del target1
    del target2
    del target3
    del metaEx
    gc.collect()

    ### Standardizing to DMSO within Plate.
    df_dmso = df[(df['Metadata_Cmpd'] == 'dmso')]
    df_dmso = df_dmso.select_dtypes(include=np.number)
    df_dmso = df_dmso.loc[:,~df_dmso.columns.str.contains('Metadata_', case=True)]
    std_values = df_dmso.std(skipna=True)
    std_values[std_values == 0] = 1
    mean_values = df_dmso.mean(skipna=True)

    df_norm = df.select_dtypes(include=np.number)
    df_norm = df_norm.loc[:,~df_norm.columns.str.contains('Metadata_', case=True)]
    df_norm = df_norm.loc[:,~df_norm.columns.str.contains('_Location_Center_', case=True)]
    df_norm = df_norm.loc[:,~df_norm.columns.str.contains('_Neighbor_', case=True)]
    df_norm = ((df_norm - mean_values) / std_values)
    df[df_norm.columns] = df_norm
    df = df.copy()

    del nuclei
    del cytoplasm
    del cells

    df.to_parquet('{}/spheroid{}SingleCell.parquet'.format(OutputDir, df['Metadata_Barcode'].unique()[0]))
    # df_scall = pd.concat([df_scall, df], ignore_index=True)
    toNpy = df.loc[:,~df.columns.str.contains('Metadata_')]
    toNpy.reset_index(inplace=True, drop=True)
    toNpy = toNpy.copy()
    dataNpy = toNpy.to_numpy()
    print(f'{df["Metadata_Barcode"].unique()[0]}has NaNs: {np.isnan(dataNpy).any()}')

    ### Mean and Median compositions
    df['Metadata_ID'] = df['Metadata_Barcode'] + '+' + df['Metadata_Well'] + '+' + df['Metadata_Cmpd']
    metadf = pd.DataFrame()
    dictMedian = { i : np.nanmedian for i in numeric_df}
    mediandf = df.groupby('Metadata_ID').agg(dictMedian).reset_index(drop=False)
    metadatacol = mediandf.pop('Metadata_ID')
    metadf[['Metadata_Barcode','Metadata_Well','Metadata_Cmpd']] = metadatacol.str.split('+', expand=True)
    mediandf.insert(0, 'Metadata_Barcode', metadf['Metadata_Barcode'])
    mediandf.insert(1, 'Metadata_Well', metadf['Metadata_Well'])
    mediandf.insert(2, 'Metadata_Cmpd', metadf['Metadata_Cmpd'])

    dictMean = { i : np.nanmean for i in numeric_df}
    meandf = df.groupby('Metadata_ID').agg(dictMean).reset_index(drop=False)
    metadatacol = meandf.pop('Metadata_ID')
    metadf[['Metadata_Barcode','Metadata_Well','Metadata_Cmpd']] = metadatacol.str.split('+', expand=True)
    meandf.insert(0, 'Metadata_Barcode', metadf['Metadata_Barcode'])
    meandf.insert(1, 'Metadata_Well', metadf['Metadata_Well'])
    meandf.insert(2, 'Metadata_Cmpd', metadf['Metadata_Cmpd'])
    del metadatacol
    del metadf
    gc.collect()

    # mediandf.to_parquet('{}/spheroid{}Median.parquet'.format(OutputDir, df['Metadata_Barcode'].unique()[0]))
    # meandf.to_parquet('{}/spheroid{}Mean.parquet'.format(OutputDir, df['Metadata_Barcode'].unique()[0]))
    # # df_meanall = pd.concat([df_meanall, meandf], ignore_index=True)
    # # df_medianll = pd.concat([df_medianall, mediandf], ignore_index=True)
    
    # # del df
    # del mediandf
    # del meandf
    gc.collect()
# df_scall.to_parquet('{}/spheroidSingleCell.parquet'.format(OutputDir))
# df_meanall.to_parquet('{}/spheroidMean.parquet'.format(OutputDir))
# df_medianall.to_parquet('{}/spheroidMedian.parquet'.format(OutputDir))

KeyboardInterrupt: 

In [ ]:
now = datetime.datetime.now()
print ('Current date and time : ')
print (now.strftime('%Y-%m-%d %H:%M:%S'))

Current date and time : 
2023-10-12 12:18:38


In [ ]:
# df_scall.head()

In [ ]:
# df_meanall.head()

In [ ]:
# toNpy = df_scall.loc[:,~df_scall.columns.str.contains('Metadata_')]
# toNpy.reset_index(inplace=True, drop=True)
# toNpy = toNpy.copy()
# dataNpy = toNpy.to_numpy()
# np.isnan(dataNpy).any()
# colnames = toNpy.columns